<a href="https://colab.research.google.com/github/HisakaKoji/bert-japanese/blob/master/notebook/finetune_to_livedoor_corpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetuning of the pretrained Japanese BERT model

Finetune the pretrained model to solve multi-class classification problems.  
This notebook requires the following objects:
- trained sentencepiece model (model and vocab files)
- pretraiend Japanese BERT model

Dataset is livedoor ニュースコーパス in https://www.rondhuit.com/download.html.  
We make test:dev:train = 2:2:6 datasets.

**This colab notebook assumes the above models are stored on some GSC bucket you can acess its objects.**

In [0]:
%cd /content

/content


In [0]:
%rm -rf bert-japanese

In [1]:
!git clone  https://github.com/HisakaKoji/bert-japanese.git

Cloning into 'bert-japanese'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 248 (delta 53), reused 33 (delta 16), pack-reused 168
Receiving objects: 100% (248/248), 310.71 KiB | 4.09 MiB/s, done.
Resolving deltas: 100% (146/146), done.


In [2]:
%cd /content/bert-japanese/

/content/bert-japanese


In [3]:
!git clone https://github.com/google-research/bert.git

Cloning into 'bert'...
remote: Enumerating objects: 333, done.
remote: Total 333 (delta 0), reused 0 (delta 0), pack-reused 333
Receiving objects: 100% (333/333), 282.45 KiB | 3.82 MiB/s, done.
Resolving deltas: 100% (183/183), done.


In [4]:
%cd ..

/content


In [5]:
!pip install -q -r bert-japanese/requirements.txt

     |████████████████████████████████| 1.0MB 3.5MB/s 


In [6]:
from google.colab import auth
auth.authenticate_user()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [7]:
!gsutil cp gs://hisaka/model/wiki-ja.model /content/bert-japanese/model/
!gsutil cp gs://hisaka/model/wiki-ja.vocab /content/bert-japanese/model/

Copying gs://hisaka/model/wiki-ja.model...
- [1 files][786.8 KiB/786.8 KiB]                                                
Operation completed over 1 objects/786.8 KiB.                                    
Copying gs://hisaka/model/wiki-ja.vocab...
- [1 files][581.7 KiB/581.7 KiB]                                                
Operation completed over 1 objects/581.7 KiB.                                    


In [8]:
%cd /content/bert-japanese/notebook

/content/bert-japanese/notebook


In [9]:
%ls

check-extract-features.ipynb       finetune-to-livedoor-corpus.ipynb
check-trained-tokenizer.ipynb      pretraining.ipynb
finetune_to_livedoor_corpus.ipynb


Check TPU.

In [49]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

TPU address is grpc://10.83.106.202:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 8697898531980608325),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 14033808939666369073),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 17853241105050111545),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 17353573357783292768),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 11419263389915717442),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 7461087437713949061),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 5711198934420970324),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 16949729568708869796),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 1109047233

## Data preparation

You need to put preprocessed data on your GCS bucket.  
To create preprocessed data, follow https://github.com/yoheikikuta/bert-japanese/blob/master/notebook/finetune-to-livedoor-corpus.ipynb.

## Finetune pre-trained model

In [25]:
!gsutil cp -r gs://hisaka/rurubu .

Copying gs://hisaka/rurubu/dev.tsv...
Copying gs://hisaka/rurubu/output/checkpoint...
\ [2 files][161.6 KiB/161.6 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://hisaka/rurubu/output/eval.tf_record...
Copying gs://hisaka/rurubu/output/eval/events.out.tfevents.1566977013.cdaa002c9f52...
Copying gs://hisaka/rurubu/output/eval/events.out.tfevents.1566978189.cdaa002c9f52...
Copying gs://hisaka/rurubu/output/eval/events.out.tfevents.1566980458.cdaa002c9f52...
Copying gs://hisaka/rurubu/output/eval/events.out.tfevents.1566982511.cdaa002c9f52...
Copying gs://hisaka/rurubu/output/eval/events.out.tfevents.1566983524.cdaa002c9f52...
Copying gs://hisaka/rurubu/output/eval/events.out.tfevents.1566989340.cdaa002c9f52...
Copying gs://

In [28]:
!ls  rurubu

dev.tsv  output  test.tsv  train.tsv


In [0]:
!cp rurubu/train.tsv  ../model/

In [0]:
!cp rurubu/test.tsv  ../model/

In [0]:
!cp rurubu/dev.tsv  ../model/

In [36]:
!git pull

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/HisakaKoji/bert-japanese
   4fb325e..109dc36  master     -> origin/master
Updating 4fb325e..109dc36
Fast-forward
 src/run_classifier.py | 4 +++-
 1 file changed, 3 insertions(+), 1 deletion(-)


In [0]:
!gsutil cp  gs://hisaka/model/model.ckpt-1400000  .

In [0]:
PRETRAINED_MODEL_PATH = 'gs://hisaka/model/model.ckpt-1400000'  # GCS bucket
INPUT_DATA_GCS = 'gs://hisaka/rurubu'  # GCS bucket
FINETUNE_OUTPUT_DIR = 'gs://hisaka/rurubu/output' # GCS bucket

In [38]:
%%time
!python3 ../src/run_classifier.py \
  --task_name=livedoor \
  --use_tpu=False \
  --tpu_name={TPU_ADDRESS} \
  --do_train=true \
  --do_eval=true \
  --data_dir=../model/ \
  --model_file=../model/wiki-ja.model \
  --vocab_file=../model/wiki-ja.vocab \
  --init_checkpoint=./rurubu/output/model.ckpt-419 \
  --max_seq_length=512 \
  --train_batch_size=64 \
  --learning_rate=2e-5 \
  --num_train_epochs=10.0 \
  --output_dir={FINETUNE_OUTPUT_DIR}




W1001 01:06:26.146538 140411182962560 deprecation_wrapper.py:119] From ../src/run_classifier.py:661: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W1001 01:06:26.146754 140411182962560 deprecation_wrapper.py:119] From ../src/run_classifier.py:661: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W1001 01:06:26.147200 140411182962560 deprecation_wrapper.py:119] From /content/bert-japanese/src/../bert/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W1001 01:06:26.147912 140411182962560 deprecation_wrapper.py:119] From ../src/run_classifier.py:682: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.

Loaded a trained SentencePiece model.
I1001 01:06:26.923278 140411182962560 utils.py:141] NumExpr defaulting to 2 threads.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, 

In [0]:
INPUT_DATA_GCS = 'gs://hisaka/mirunavi2'  # GCS bucket

In [44]:
!gsutil cp -r gs://hisaka/mirunavi2  .

Copying gs://hisaka/mirunavi2/test.tsv...
- [1 files][  5.1 MiB/  5.1 MiB]                                                
Operation completed over 1 objects/5.1 MiB.                                      


In [45]:
!ls

check-extract-features.ipynb	   finetune-to-livedoor-corpus.ipynb
check-trained-tokenizer.ipynb	   mirunavi2
{FINETUNE_OUTPUT_DIR}		   pretraining.ipynb
finetune_to_livedoor_corpus.ipynb  rurubu


In [42]:
INPUT_DATA_GCS

'gs://hisaka/mirunavi2'

## Predict using the finetuned model

Let's predict test data using the finetuned model.  

In [50]:
%%time
!python3 ../src/run_classifier.py \
  --task_name=livedoor \
  --use_tpu=True \
  --tpu_name={TPU_ADDRESS} \
  --do_train=false \
  --do_eval=false \
  --do_predict=true \
  --data_dir=mirunavi2 \
  --model_file=../model/wiki-ja.model \
  --vocab_file=../model/wiki-ja.vocab \
  --init_checkpoint=./rurubu/output/model.ckpt-419 \
  --max_seq_length=512 \
  --train_batch_size=64 \
  --learning_rate=2e-5 \
  --num_train_epochs=3.0 \
  --output_dir={FINETUNE_OUTPUT_DIR}




W1001 09:35:09.778386 139634875082624 deprecation_wrapper.py:119] From ../src/run_classifier.py:661: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W1001 09:35:09.778635 139634875082624 deprecation_wrapper.py:119] From ../src/run_classifier.py:661: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W1001 09:35:09.779159 139634875082624 deprecation_wrapper.py:119] From /content/bert-japanese/src/../bert/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W1001 09:35:09.780067 139634875082624 deprecation_wrapper.py:119] From ../src/run_classifier.py:682: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.

Loaded a trained SentencePiece model.
I1001 09:35:12.154394 139634875082624 utils.py:141] NumExpr defaulting to 2 threads.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, 

## Evaluation

Download result and original data.

In [54]:
!gsutil cp {FINETUNE_OUTPUT_DIR}/test_results.tsv .
!gsutil cp {INPUT_DATA_GCS}/train.tsv .
!gsutil cp {INPUT_DATA_GCS}/dev.tsv .
!gsutil cp {INPUT_DATA_GCS}/test.tsv .

Copying gs://hisaka/rurubu/output/test_results.tsv...
- [1 files][  6.4 MiB/  6.4 MiB]                                                
Operation completed over 1 objects/6.4 MiB.                                      
CommandException: No URLs matched: gs://hisaka/mirunavi2/train.tsv
CommandException: No URLs matched: gs://hisaka/mirunavi2/dev.tsv
Copying gs://hisaka/mirunavi2/test.tsv...
/ [1 files][  5.1 MiB/  5.1 MiB]                                                
Operation completed over 1 objects/5.1 MiB.                                      


### Trained model

Check accuracy.

In [0]:
import numpy as np
import pandas as pd

In [60]:
import sys
sys.path.append("../src")

from run_classifier import LivedoorProcessor

processor = LivedoorProcessor()
label_list = processor.get_labels()

In [0]:
result = pd.read_csv("./test_results.tsv", sep='\t', header=None)

In [56]:
result.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,0.892444,0.005709,0.011875,0.003597,0.002588,0.003702,0.008756,0.034030,0.010379,0.001593,0.002533,0.000703,0.002605,0.000494,0.000369,0.001027,0.000794,0.000347,0.000474,0.000625,0.000477,0.000690,0.000354,0.000342,0.000643,0.007265,0.001547,0.000722,0.003023,0.000295
1,0.000702,0.000487,0.000458,0.000757,0.000860,0.000384,0.987598,0.000947,0.000543,0.000323,0.001172,0.000415,0.000412,0.000295,0.000250,0.000417,0.000268,0.000237,0.000415,0.000192,0.000220,0.000243,0.000113,0.000279,0.000229,0.000463,0.000553,0.000264,0.000333,0.000173


In [0]:
test_df = pd.read_csv("./test.tsv", sep='\t')

In [0]:
test_df['predict'] = [ label_list[np.array(elem[1]).argmax()] for elem in result.iterrows() ]

In [62]:
sum( test_df['label'] == test_df['predict'] ) / len(test_df)

0.03795826283135928

In [0]:
### 1/5 of full training data.
# sum( test_df['label'] == test_df['predict'] ) / len(test_df)

In [0]:
test_df.to_csv('./test20191001.csv')

A littel more detailed check using `sklearn.metrics`.

In [0]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [0]:
print(classification_report(test_df['label'], test_df['predict']))

                                        precision    recall  f1-score   support

                   Agricultural ritual       0.00      0.00      0.00         2
                 Procession-and-parade       0.00      0.00      0.00         9
                    Shrine-floats-etc.       0.00      0.00      0.00        44
Traditional-Festivalsand-annual-events       0.39      0.76      0.52        51
 Traditional-performing-arts-and-dance       0.58      0.74      0.65        62
                                animal       0.00      0.00      0.00         1
                              festival       0.00      0.00      0.00         2
                                  fire       0.20      0.06      0.09        35
                             fireworks       0.79      0.98      0.87       108
                         flower-nature       0.11      0.50      0.18         2
                                  food       0.00      0.00      0.00         7
                          illumination 

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [0]:
### 1/5 of full training data.
# print(classification_report(test_df['label'], test_df['predict']))

In [0]:
print(confusion_matrix(test_df['label'], test_df['predict']))

[[ 0  0  1  0  0  0  0  1  0  0  0  0  0  0  0  0  0]
 [ 0  1  3  3  0  0  0  0  2  0  0  0  0  0  0  0  0]
 [ 0  0 19 14  8  0  0  3  0  0  0  0  0  0  0  0  0]
 [ 0  0  3 39  2  0  0  4  2  1  0  0  0  0  0  0  0]
 [ 0  0  6 13 37  0  0  4  1  0  0  0  0  0  0  0  1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0]
 [ 0  0  0  0  0  0  0  0  0  1  0  0  1  0  0  0  0]
 [ 0  2  5  8  1  0  0 17  1  1  0  0  0  0  0  0  0]
 [ 0  0  3  3  0  0  1  2 93  2  0  2  1  0  1  0  0]
 [ 0  0  0  1  0  0  0  0  0  1  0  0  0  0  0  0  0]
 [ 0  0  1  0  0  0  1  1  0  1  1  0  1  0  0  1  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  5  0  0  0  0  0]
 [ 0  0  0  1  0  0  0  0  0  0  0  0  2  0  0  0  0]
 [ 0  0  0  0  0  0  2  0  0  0  0  1  0  0  0  0  1]
 [ 0  0  0  1  0  0  2  1  0  0  0  1  0  0  0  0  1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]]


In [0]:
### 1/5 of full training data.
# print(confusion_matrix(test_df['label'], test_df['predict']))

### Simple baseline model.

In [0]:
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [0]:
train_df = pd.read_csv("./train.tsv", sep='\t')
dev_df = pd.read_csv("./dev.tsv", sep='\t')
test_df = pd.read_csv("./test.tsv", sep='\t')

In [0]:
!apt-get -q install -y mecab libmecab-dev mecab-ipadic mecab-ipadic-utf8

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libmecab2 mecab-utils
The following NEW packages will be installed:
  libmecab-dev libmecab2 mecab mecab-ipadic mecab-ipadic-utf8 mecab-utils
0 upgraded, 6 newly installed, 0 to remove and 14 not upgraded.
Need to get 12.8 MB of archives.
After this operation, 60.4 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libmecab2 amd64 0.996-5 [257 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libmecab-dev amd64 0.996-5 [308 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 mecab-utils amd64 0.996-5 [4,856 B]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 mecab-ipadic all 2.7.0-20070801+main-1 [12.1 MB]
Get:5 http://archive.ubuntu.com/ubuntu bionic/universe amd64 mecab amd64 0.996-5 [132 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic/universe amd64 mecab-

In [0]:
!pip install -q mecab-python3==0.7

     |████████████████████████████████| 51kB 2.4MB/s 


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import GradientBoostingClassifier
import MeCab

In [0]:
m = MeCab.Tagger("-Owakati")

In [0]:
train_dev_df = pd.concat([train_df, dev_df])

In [0]:
train_dev_xs = train_dev_df['text'].apply(lambda x: m.parse(x))
train_dev_ys = train_dev_df['label']

test_xs = test_df['text'].apply(lambda x: m.parse(x))
test_ys = test_df['label']

In [0]:
vectorizer = TfidfVectorizer(max_features=750)
train_dev_xs_ = vectorizer.fit_transform(train_dev_xs)
test_xs_ = vectorizer.transform(test_xs)

The following set up is not exactly identical to that of BERT because inside Classifier it uses `train_test_split` with shuffle.  
In addition, parameters are not well tuned, however, we think it's enough to check the power of BERT.

In [0]:
%%time

model = GradientBoostingClassifier(n_estimators=200,
                                   validation_fraction=len(dev_df)/len(train_df),
                                   n_iter_no_change=5,
                                   tol=0.01,
                                   random_state=23)

### 1/5 of full training data.
# model = GradientBoostingClassifier(n_estimators=200,
#                                    validation_fraction=len(train_df)/len(dev_df),
#                                    n_iter_no_change=5,
#                                    tol=0.01,
#                                    random_state=23)

model.fit(train_dev_xs_, train_dev_ys)

In [0]:
print(classification_report(test_ys, model.predict(test_xs_)))

In [0]:
### 1/5 of full training data.
# print(classification_report(test_ys, model.predict(test_xs_)))

                precision    recall  f1-score   support

dokujo-tsushin       0.82      0.71      0.76       178
  it-life-hack       0.86      0.88      0.87       172
 kaden-channel       0.91      0.87      0.89       176
livedoor-homme       0.67      0.63      0.65        95
   movie-enter       0.87      0.95      0.91       158
        peachy       0.70      0.78      0.73       174
          smax       1.00      1.00      1.00       167
  sports-watch       0.87      0.95      0.91       190
    topic-news       0.92      0.82      0.87       163

     micro avg       0.85      0.85      0.85      1473
     macro avg       0.85      0.84      0.84      1473
  weighted avg       0.86      0.85      0.85      1473



In [0]:
print(confusion_matrix(test_ys, model.predict(test_xs_)))

[[153   4   1   4   2  13   0   1   0]
 [  3 155   6   3   0   1   0   2   2]
 [  0   5 165   0   2   0   1   1   2]
 [  3   4   6  70   4   6   0   1   1]
 [  1   0   1   3 152   1   0   0   0]
 [  7   1   1   2   3 160   0   0   0]
 [  0   0   0   0   0   0 167   0   0]
 [  1   0   0   2   0   0   0 186   1]
 [  3   1   3   5   0   3   0   8 140]]


In [0]:
### 1/5 of full training data.
# print(confusion_matrix(test_ys, model.predict(test_xs_)))

[[127   6   1   4   5  29   0   5   1]
 [  2 152   6   7   1   2   0   1   1]
 [  1   8 153   4   5   3   0   1   1]
 [  3   7   3  60   3  15   0   2   2]
 [  1   0   1   1 150   5   0   0   0]
 [ 15   3   0  10   8 135   0   2   1]
 [  0   0   0   0   0   0 167   0   0]
 [  1   0   2   0   0   1   0 181   5]
 [  5   0   2   3   0   4   0  15 134]]
